# Deep Learning Tweets

### Importing Libraries

In [9]:
import _pickle as pickle
from sklearn.model_selection import train_test_split
import pandas as pd
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Input, Dense, LSTM, Embedding
from keras.layers import Dropout, Activation, Bidirectional, GlobalMaxPool1D
from keras.models import Model
from keras import initializers, regularizers, constraints, optimizers, layers
from keras.preprocessing import text, sequence

### Loading the data

In [5]:
# Loading in the DF
with open("main_df.pkl",'rb') as fp:
    main_df = pickle.load(fp)

# Loading in the cleaned tweet data
with open("clean_tweets.pkl",'rb') as fp:
    data = pickle.load(fp)

### Train, test, split

In [7]:
X, y = data, main_df.City

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [10]:
y = pd.get_dummies(y).values

In [11]:
tokenizer = text.Tokenizer(num_words=1000)
tokenizer.fit_on_texts(X)
list_tokenized_tweets = tokenizer.texts_to_sequences(X)
X_t = sequence.pad_sequences(list_tokenized_tweets, maxlen=100)

In [12]:
embedding_size = 500
input_ = Input(shape=(100,))
x = Embedding(1000, embedding_size)(input_)
x = LSTM(50, return_sequences=True)(x)
x = GlobalMaxPool1D()(x)
x = Dropout(0.5)(x)
x = Dense(50, activation='relu')(x)
x = Dropout(0.5)(x)
# There are 2 different possible classes, so we use 2 neurons in our output layer
x = Dense(2, activation='softmax')(x)

model = Model(inputs=input_, outputs=x)

In [13]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [14]:
model.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 100)               0         
_________________________________________________________________
embedding_1 (Embedding)      (None, 100, 500)          500000    
_________________________________________________________________
lstm_1 (LSTM)                (None, 100, 50)           110200    
_________________________________________________________________
global_max_pooling1d_1 (Glob (None, 50)                0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 50)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 50)                2550      
_________________________________________________________________
dropout_2 (Dropout)          (None, 50)                0   

In [15]:
model.fit(X_t, y, epochs=3, batch_size=2000, validation_split=0.2)

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where

Train on 16000 samples, validate on 4000 samples
Epoch 1/3
16000/16000 [==============================] - 32s 2ms/step - loss: 0.6780 - accuracy: 0.5892 - val_loss: 0.9527 - val_accuracy: 0.0000e+00
Epoch 2/3
16000/16000 [==============================] - 27s 2ms/step - loss: 0.6675 - accuracy: 0.6234 - val_loss: 0.8753 - val_accuracy: 0.0000e+00
Epoch 3/3
16000/16000 [==============================] - 27s 2ms/step - loss: 0.6637 - accuracy: 0.6244 - val_loss: 0.9232 - val_accuracy: 0.0000e+00
